##### References:
* https://www.youtube.com/watch?v=YCzL96nL7j0&list=PLblh5JKOoLUIxGDQs4LFFD--41Vzf-ME1&index=18&ab_channel=StatQuestwithJoshStarmer
* https://wandb.ai/sauravmaheshkar/LSTM-PyTorch/reports/Using-LSTM-in-PyTorch-A-Tutorial-With-Examples--VmlldzoxMDA2NTA5
* https://stackoverflow.com/questions/48302810/whats-the-difference-between-hidden-and-output-in-pytorch-lstm

In [1]:
# LSTM theory, and concept
# LSTM implementation in PyTorch for sentiment classification and spell checker problem
# what is xLSTM ?  A new paper came out on 7th may, 2024

In [2]:
# LSTM for SMS Spam classification

# 3 Methods to generate / embeddings:
# 1. use word2vec to train your embeddings [used in this code]
# 2. Use existing embedding like glove
# 3. user BERT embeddings


# 4 methods for ML modelling
# 1. use the embeddings as is and train a tradition ML model like RF, GBM, etc. 
# 2. Use the embeddng as is and train a Neural Network model but not embeddings (linear layer, lstm layer,etc.)
# 3. Use the embeddings and finetune/retrain them during training process of Neural Network [used in this code]
# 4. Use BERTforClassification model which use BERT embeddings and cls token of BERT model to classfiy

# In this section, we will use, embedding (1st option) and modelling (3rd option) combination

In [3]:
import pandas as pd
import os

os.getcwd()

'c:\\Users\\manpresingh\\OneDrive - Microsoft\\Personal\\PyTorch_and_Advanced NLP'

In [4]:
data = pd.read_csv('SMSSpamCollection.tsv', sep='\t', header=None)

In [5]:
data.rename(columns={0:'target',1:'input'}, inplace=True)

In [6]:
data.head()

,target,input
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data.input, 
                                                    data.target, 
                                                    test_size=.2, 
                                                    stratify=data.target,
                                                    random_state=100
                                                    )

In [9]:
X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

In [10]:
X_train_df = pd.DataFrame(X_train, columns=['input'])
X_test_df = pd.DataFrame(X_test, columns=['input'])
X_train_df

,input
0,Wrong phone! This phone! I answer this one but...
1,When ü login dat time... Dad fetching ü home now?
2,A guy who gets used but is too dumb to realize...
3,"Hey sweet, I was wondering when you had a mome..."
4,Y lei?
...,...
4449,Yes :)it completely in out of form:)clark also...
4450,"Alright omw, gotta change my order to a half8th"
4451,Ranjith cal drpd Deeraj and deepak 5min hold
4452,I have to take exam with in march 3


In [11]:
import gensim

In [12]:
# text cleaning:

def simple_prep_process( data): ## lowercases, tokenizes, and de-accents the text
        return data.apply(lambda x:  gensim.utils.simple_preprocess(x, deacc=True))

In [13]:
X_train_df['cleaned_input'] =  simple_prep_process(X_train_df.input)
X_test_df['cleaned_input'] =  simple_prep_process(X_test_df.input)

In [14]:
X_train_df

,input,cleaned_input
0,Wrong phone! This phone! I answer this one but...,"[wrong, phone, this, phone, answer, this, one,..."
1,When ü login dat time... Dad fetching ü home now?,"[when, login, dat, time, dad, fetching, home, ..."
2,A guy who gets used but is too dumb to realize...,"[guy, who, gets, used, but, is, too, dumb, to,..."
3,"Hey sweet, I was wondering when you had a mome...","[hey, sweet, was, wondering, when, you, had, m..."
4,Y lei?,[lei]
...,...,...
4449,Yes :)it completely in out of form:)clark also...,"[yes, it, completely, in, out, of, form, clark..."
4450,"Alright omw, gotta change my order to a half8th","[alright, omw, gotta, change, my, order, to, h..."
4451,Ranjith cal drpd Deeraj and deepak 5min hold,"[ranjith, cal, drpd, deeraj, and, deepak, min,..."
4452,I have to take exam with in march 3,"[have, to, take, exam, with, in, march]"


In [15]:
from gensim.models import Word2Vec, Doc2Vec

In [16]:
embed_size=300
w2v_model = Word2Vec(sentences = X_train_df['cleaned_input'],
         window=7, min_count=1,
         sg=1,
         vector_size=embed_size)

In [17]:
len(w2v_model.wv.key_to_index)

6901

In [18]:
w2v_model.wv.key_to_index

{'to': 0,
 'you': 1,
 'the': 2,
 'and': 3,
 'in': 4,
 'is': 5,
 'me': 6,
 'my': 7,
 'it': 8,
 'for': 9,
 'your': 10,
 'of': 11,
 'call': 12,
 'that': 13,
 'have': 14,
 'on': 15,
 'now': 16,
 'are': 17,
 'can': 18,
 'but': 19,
 'so': 20,
 'not': 21,
 'we': 22,
 'or': 23,
 'ur': 24,
 'do': 25,
 'if': 26,
 'will': 27,
 'at': 28,
 'get': 29,
 'be': 30,
 'with': 31,
 'no': 32,
 'just': 33,
 'this': 34,
 'gt': 35,
 'lt': 36,
 'when': 37,
 'ok': 38,
 'how': 39,
 'up': 40,
 'go': 41,
 'what': 42,
 'from': 43,
 'free': 44,
 'out': 45,
 'know': 46,
 'all': 47,
 'll': 48,
 'am': 49,
 'then': 50,
 'day': 51,
 'like': 52,
 'he': 53,
 'good': 54,
 'got': 55,
 'was': 56,
 'come': 57,
 'there': 58,
 'its': 59,
 'only': 60,
 'time': 61,
 'love': 62,
 'text': 63,
 'send': 64,
 'want': 65,
 'txt': 66,
 'need': 67,
 'by': 68,
 'home': 69,
 'as': 70,
 'one': 71,
 'she': 72,
 'going': 73,
 'don': 74,
 'lor': 75,
 'stop': 76,
 'still': 77,
 'today': 78,
 'da': 79,
 'our': 80,
 'dont': 81,
 'see': 82,
 'sorry

In [19]:
w2v_model.wv.index_to_key

['to',
 'you',
 'the',
 'and',
 'in',
 'is',
 'me',
 'my',
 'it',
 'for',
 'your',
 'of',
 'call',
 'that',
 'have',
 'on',
 'now',
 'are',
 'can',
 'but',
 'so',
 'not',
 'we',
 'or',
 'ur',
 'do',
 'if',
 'will',
 'at',
 'get',
 'be',
 'with',
 'no',
 'just',
 'this',
 'gt',
 'lt',
 'when',
 'ok',
 'how',
 'up',
 'go',
 'what',
 'from',
 'free',
 'out',
 'know',
 'all',
 'll',
 'am',
 'then',
 'day',
 'like',
 'he',
 'good',
 'got',
 'was',
 'come',
 'there',
 'its',
 'only',
 'time',
 'love',
 'text',
 'send',
 'want',
 'txt',
 'need',
 'by',
 'home',
 'as',
 'one',
 'she',
 'going',
 'don',
 'lor',
 'stop',
 'still',
 'today',
 'da',
 'our',
 'dont',
 'see',
 'sorry',
 'reply',
 'about',
 'hi',
 'back',
 'tell',
 'later',
 'new',
 'mobile',
 'her',
 'they',
 'take',
 'please',
 'pls',
 'dear',
 'has',
 'did',
 'some',
 'any',
 'been',
 'think',
 'phone',
 'week',
 'happy',
 'well',
 'here',
 'much',
 'him',
 'who',
 're',
 'claim',
 'night',
 'msg',
 'oh',
 'an',
 'more',
 'hope',


In [35]:
w2v_model.wv.key_to_index['how']

39

In [47]:
w2v_model.wv.get_vector(39, norm=True)[0:10]

array([-0.02847072,  0.02612918,  0.04232004, -0.00957793, -0.00946878,
       -0.04808065,  0.0783463 ,  0.15326379, -0.05204922,  0.00731871],
      dtype=float32)

In [45]:
w2v_model.wv.get_mean_vector([39])[0:10]

array([-0.02847072,  0.02612918,  0.04232004, -0.00957793, -0.00946878,
       -0.04808065,  0.0783463 ,  0.15326379, -0.05204922,  0.00731871],
      dtype=float32)

In [22]:
w2v_model.wv.most_similar('it')

[('sure', 0.9908840656280518),
 ('would', 0.9888841509819031),
 ('gonna', 0.9868761897087097),
 ('here', 0.9856452941894531),
 ('some', 0.9853847622871399),
 ('work', 0.984980583190918),
 ('so', 0.9849373698234558),
 ('yeah', 0.9839804768562317),
 ('something', 0.9838212728500366),
 ('one', 0.983314573764801)]

In [23]:
X_train_df.cleaned_input[0]

['wrong',
 'phone',
 'this',
 'phone',
 'answer',
 'this',
 'one',
 'but',
 'assume',
 'the',
 'other',
 'is',
 'people',
 'don',
 'well']

In [24]:
w2v_model.wv.get_mean_vector(X_train_df.cleaned_input[0])
# This represents the mean embedding of 1st sentence.
# this is useful when you want to use the embedding for an ML model or DNN model without finetuning of embeddings

array([-0.01866138,  0.09088621,  0.01010347, -0.01167167, -0.00278436,
       -0.0697023 ,  0.09133922,  0.15966056, -0.01450161, -0.03075367,
        0.06160251, -0.04894392, -0.03545335,  0.00900614, -0.05132652,
       -0.05500349,  0.06946038,  0.03673387, -0.01017637, -0.05604023,
       -0.00239395,  0.00534344,  0.07115864,  0.04612632,  0.04577341,
        0.00589511, -0.09053893,  0.01028949, -0.01969096, -0.112578  ,
        0.03971498, -0.02108231,  0.01643392,  0.02529233, -0.03024879,
        0.06300761,  0.02645947, -0.10853554, -0.00486615, -0.022777  ,
       -0.00989388, -0.01027509, -0.01488046, -0.08992512,  0.01419432,
        0.13474324,  0.0431227 ,  0.01788112, -0.02632701,  0.06212071,
        0.05584181,  0.00929834, -0.02039016,  0.05297269, -0.02612629,
        0.11144539,  0.03306751,  0.00798122,  0.01045389, -0.00513319,
       -0.02109441, -0.03163652,  0.01357193,  0.00659585, -0.04944506,
        0.03700563, -0.05094394, -0.01518382, -0.04460403,  0.00

In [25]:
X_train_df.shape

(4454, 2)

In [26]:
w2v_model.wv.get_mean_vector(['jhndfskjvbgkd'])
# for unknown words, the return embedding is 0 

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [26]:
X_train_df.shape

(4454, 2)

In [27]:
import numpy as np

In [28]:
# # This can be used to generate embedding for the entire data

# %%time
# embedding_list=[]
# for i in range(0, len(X_train_df)):
#     try:
#         embedding_list.append(w2v_model.wv.get_mean_vector(X_train_df.cleaned_input[i]))
#     except:
#         print(i)
#         embedding_list.append(np.random.normal(scale=.6,size=(embed_size,)))
#         # scale represents the Standard deviation of the data
#         # larger value results in a winder distribution of values

In [51]:
train_vocab = w2v_model.wv.key_to_index
max_len = len(train_vocab)
print(max_len)
word_embedding_matrix =  np.zeros((max_len+2, embed_size))

words_found = 0

for i, word in enumerate(train_vocab):
    try:
        word_embedding_matrix[i]=w2v_model.wv.get_vector(word)
        words_found+=1
    except:
        word_embedding_matrix[i] = np.random.random(scale=.6, size=(embed_size,))

6901


In [53]:
words_found
# since all the words were found.
# it may happen that in case of glove or any other pretrained embedding, some of our vocab words may not get the embedding
# So, then we initalise it with a random number and those embeddings get trained in the model

6901

In [55]:
len(word_embedding_matrix)
# 2 tokens are special tokens:
# 1 is for unknown words 
# 2 is for padding

6903

In [83]:
word_embedding_matrix[6900]

array([-6.21277979e-03,  8.38991851e-02, -8.28249205e-04, -4.18483745e-03,
        1.51266868e-03, -5.38781956e-02,  6.82090521e-02,  1.16374984e-01,
        4.36041597e-03, -1.99522320e-02,  1.85837932e-02, -5.48968576e-02,
       -1.47887031e-02, -1.10565661e-03, -4.86317165e-02, -2.93762702e-02,
        4.66998145e-02,  2.70344801e-02,  1.27363075e-02, -2.18607821e-02,
       -2.27865595e-02,  1.23099759e-02,  2.31501888e-02,  1.80544928e-02,
        4.54181284e-02,  1.27840657e-02, -5.40392436e-02,  5.29623963e-03,
       -3.30058001e-02, -8.02401975e-02,  2.74393149e-02, -1.32855559e-02,
        3.13437870e-03,  9.82671324e-03, -1.67037901e-02,  2.83521116e-02,
        2.23893095e-02, -7.78076500e-02,  7.14191888e-03, -1.31827826e-02,
       -9.88247432e-03, -4.13602777e-03, -1.95401404e-02, -5.98052777e-02,
        4.40205354e-03,  7.86848143e-02,  2.76795030e-02,  4.11512069e-02,
       -2.53173020e-02,  5.25153242e-02,  3.26507427e-02, -7.73164723e-03,
       -2.06406433e-02,  

In [84]:
word_embedding_matrix[6901:]
# last 2 are zero as they are for unknown words and padding

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [71]:
def word_tokens(sentence):
    sent_tok=[]
    for word in sentence:
        try:
            sent_tok.append(train_vocab[word])

        except:
            sent_tok.append(6901) # for Unkown words
    return sent_tok

In [72]:

sentence_tok=[]
sentence_tok=[word_tokens(sentence) for sentence in X_train_df.cleaned_input];

In [73]:
sentence_tok

[[754, 104, 34, 104, 562, 34, 71, 19, 2028, 2, 210, 5, 274, 74, 107],
 [37, 1315, 264, 61, 476, 3464, 69, 16],
 [473, 111, 966, 717, 19, 5, 121, 2693, 0, 2669, 8],
 [125,
  310,
  56,
  952,
  37,
  1,
  133,
  1002,
  26,
  1,
  356,
  57,
  0,
  6,
  65,
  0,
  64,
  1733,
  0,
  189,
  19,
  8,
  137,
  41,
  181,
  1546,
  9,
  161,
  303,
  850,
  1534,
  1101,
  438,
  37,
  1,
  662,
  40,
  13,
  1252,
  9,
  6,
  0,
  41,
  0,
  3,
  653,
  2,
  2372,
  3553,
  223,
  1,
  88,
  6,
  39,
  0,
  25,
  13,
  23,
  25,
  1,
  46,
  100,
  210,
  128,
  0,
  653,
  374,
  2399,
  303,
  93,
  18,
  653,
  257,
  1206,
  43,
  2,
  1199,
  101,
  186,
  168,
  30,
  123,
  7,
  2312,
  1222,
  369],
 [465],
 [26, 21, 248, 47, 624, 50, 48, 41, 69, 75, 26, 232, 191, 52, 781, 8, 38],
 [103, 236, 9, 2, 245, 380, 2137, 6, 37, 1, 29, 58, 26, 1, 357, 29, 58],
 [97, 27, 12, 3453, 96, 3447],
 [158, 9, 397, 45, 9, 6, 143, 1106],
 [3433,
  24,
  643,
  5,
  15,
  2,
  128,
  126,
  321,
  217

In [74]:
sentence_tok[0]

[754, 104, 34, 104, 562, 34, 71, 19, 2028, 2, 210, 5, 274, 74, 107]

In [75]:
X_train_df.cleaned_input[0]

['wrong',
 'phone',
 'this',
 'phone',
 'answer',
 'this',
 'one',
 'but',
 'assume',
 'the',
 'other',
 'is',
 'people',
 'don',
 'well']

In [76]:
for word in X_train_df.cleaned_input[0]:
    print(w2v_model.wv.key_to_index[word])

754
104
34
104
562
34
71
19
2028
2
210
5
274
74
107


In [126]:

import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader

In [127]:
text_list=[]
for sent_tok in sentence_tok:
    processed_text = torch.tensor(sent_tok, dtype=torch.int64)
    text_list.append(processed_text)

In [128]:

#padding
text_list = pad_sequence(text_list, batch_first=True, padding_value=6902)

In [129]:
text_list

tensor([[ 754,  104,   34,  ..., 6902, 6902, 6902],
        [  37, 1315,  264,  ..., 6902, 6902, 6902],
        [ 473,  111,  966,  ..., 6902, 6902, 6902],
        ...,
        [2176, 1172, 2704,  ..., 6902, 6902, 6902],
        [  14,    0,   94,  ..., 6902, 6902, 6902],
        [   7,  262,  237,  ..., 6902, 6902, 6902]])

In [130]:
# Dataset and DataLoader

In [131]:
y_train_num = np.where(y_train=='ham',0,1)

In [132]:

train_embedding_dataloader = DataLoader(
list(
    zip(
        text_list,
        torch.tensor(y_train_num, dtype=torch.float32)
        )
    ),
batch_size=64
    
)


In [133]:
# train_vocab, embed_size, word_embedding_matrix
# vocab_size = word_embedding_matrix.shape[0]


In [374]:
class myTextClassificationModel(torch.nn.Module):
    
    def __init__(self, word_embedding_matrix_tensor, vocab_size, embed_size, hidden_size):
        super(myTextClassificationModel, self).__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_size, sparse=False, mode='mean')
        self.embedding.from_pretrained(word_embedding_matrix_tensor)
        self.lstm_block_1 = torch.nn.LSTM(embed_size, hidden_size, num_layers=2, batch_first=True,
                                          dropout=0.2,  bidirectional=False)
        #  When you increase num_layers, you're stacking multiple LSTMs on top of each other, creating a "deep" LSTM
        # num_layers=2 means you have a 2-layer LSTM. The first LSTM layer processes the input sequence and produces a new sequence, which is then processed by the second LSTM layer
        self.linear1 = torch.nn.Linear(hidden_size, int(hidden_size/2))
        self.relu1 = torch.nn.ReLU()
        self.batchNorm = torch.nn.BatchNorm1d(int(hidden_size/2))
        self.linear2 = torch.nn.Linear(int(hidden_size/2), 1)
        

    def forward(self, x):
        self.embedd = self.embedding(x)
        self.lstm_output, (self.hidden_state, self.cell_state) = self.lstm_block_1(self.embedd)
        # self.lstm_out, _ = self.lstm_block_1(self.embedd)
        out = self.linear1(self.lstm_output)
        out = self.relu1(out)
        out = self.linear2(out)
        y_pred = torch.sigmoid(out)
        return y_pred
        
        

In [375]:
word_embedding_matrix_tensor = torch.tensor(word_embedding_matrix, dtype=torch.float32)

In [376]:
model = myTextClassificationModel(word_embedding_matrix_tensor=word_embedding_matrix_tensor, 
                                  vocab_size=word_embedding_matrix.shape[0],
                                  embed_size=embed_size,
                                  hidden_size=1024)

In [377]:
model

myTextClassificationModel(
  (embedding): EmbeddingBag(6903, 300, mode='mean')
  (lstm_block_1): LSTM(300, 1024, num_layers=2, batch_first=True, dropout=0.2)
  (linear1): Linear(in_features=1024, out_features=512, bias=True)
  (relu1): ReLU()
  (batchNorm): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=512, out_features=1, bias=True)
)

In [378]:
# hidden_size = 256
learning_rate = .001
num_epochs=15
n_total_steps = len(train_embedding_dataloader)
print(n_total_steps)

70


In [379]:
criterion = torch.nn.BCELoss() ## Binary Cross Entropy Loss
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [380]:
count = 0
# epochs=2

for epoch in range(num_epochs):
    myloss = []
    for i, (sentence, label) in enumerate(train_embedding_dataloader):
        # sentence = sentence.unsqueeze(0)
        count+=1
        y_pred = model(sentence)
         
        loss = criterion(torch.reshape(y_pred,(-1,)), label)
         
        optimizer.zero_grad()
         
        loss.backward()
         
        optimizer.step()
        
        myloss.append(round(loss.item(),5))
    print(f'Epoch {epoch+1} of {num_epochs}; Mean_Loss =  {round(np.mean(myloss),8)}')
         

Epoch 1 of 15; Mean_Loss =  0.40879414
Epoch 2 of 15; Mean_Loss =  0.39593029
Epoch 3 of 15; Mean_Loss =  0.39264314
Epoch 4 of 15; Mean_Loss =  0.36022157
Epoch 5 of 15; Mean_Loss =  0.23802557
Epoch 6 of 15; Mean_Loss =  0.14521986
Epoch 7 of 15; Mean_Loss =  0.09463357
Epoch 8 of 15; Mean_Loss =  0.07841771
Epoch 9 of 15; Mean_Loss =  0.06850786
Epoch 10 of 15; Mean_Loss =  0.05899486
Epoch 11 of 15; Mean_Loss =  0.05521286
Epoch 12 of 15; Mean_Loss =  0.04105114
Epoch 13 of 15; Mean_Loss =  0.04326871
Epoch 14 of 15; Mean_Loss =  0.04478686
Epoch 15 of 15; Mean_Loss =  0.03654


In [384]:
label.shape, torch.reshape(y_pred,(-1,)).shape

(torch.Size([38]), torch.Size([38]))

In [385]:
from sklearn.metrics import roc_auc_score

In [387]:
 torch.reshape(y_pred,(-1,))

tensor([1.4641e-02, 3.0202e-03, 3.3384e-04, 9.9966e-01, 9.0145e-01, 6.5512e-05,
        1.5835e-03, 1.2907e-02, 5.3364e-04, 2.0332e-04, 1.6339e-04, 3.8134e-02,
        2.5747e-04, 4.7305e-05, 3.3635e-04, 5.6245e-04, 1.2548e-02, 4.8085e-03,
        8.7927e-03, 9.9684e-01, 3.4609e-05, 1.5561e-04, 2.6531e-04, 1.2299e-04,
        1.5083e-06, 1.1452e-02, 2.8797e-03, 2.6983e-01, 9.9999e-01, 2.1630e-05,
        3.3938e-02, 1.0243e-04, 1.2584e-03, 4.8467e-04, 7.1791e-02, 2.2318e-03,
        1.2489e-02, 7.8242e-03], grad_fn=<ViewBackward0>)

In [389]:
roc_auc_score(label, pd.DataFrame( torch.reshape(y_pred,(-1,)).detach().numpy()))

1.0

In [383]:
model.lstm_output.shape,model.hidden_state.shape,model.cell_state.shape

(torch.Size([38, 1024]), torch.Size([2, 1024]), torch.Size([2, 1024]))

In [ ]:
# Test model performance on Test data: